# Validation

In order to validate our results, this notebook will show the flood mapping for the same area and time range from the ['Global Flood Monitoring'(GFM)](https://services.eodc.eu/browser/#/v1/collections/GFM) to allow comparisons.

In [ ]:
import hvplot.xarray
import pystac_client
from odc import stac as odc_stac
from dask.distributed import Client, wait

client = Client(processes=False, threads_per_worker=2, n_workers=3, memory_limit="12GB")

# Connect to STAC catalog
eodc_catalog = pystac_client.Client.open("https://stac.eodc.eu/api/v1/")

# Search for available images
time_range = "2022-10-11/2022-10-25"
bounding_box = [12.3, 54.3, 13.1, 54.6]
search = eodc_catalog.search(collections="GFM", bbox=bounding_box, datetime=time_range)
items_GFM = search.item_collection()

print(f"Found {len(items_GFM)} items") 

### Load the data

In [ ]:
import pyproj
crs = pyproj.CRS.from_wkt(items_GFM[0].properties["proj:wkt2"])

# Set the resolution of the data
resolution = items_GFM[0].properties['gsd']

GFM_dc= odc_stac.load(
    items_GFM, 
    bbox=bounding_box,   # Define the bounding box for the area of interest
    crs=crs,   # Set the coordinate reference system
    bands=["tuw_likelihood","tuw_flood_extent"],   # Specify the bands to load
    resolution=resolution,   # Set the resolution of the data
    dtype='uint8',   # Define the data type
    chunks={"x": 1000, "y": 1000, "time": -1},  # Set the chunk size for Dask
)



### Pre-processing 

In [ ]:
GFM_dc["tuw_flood_extent"] = GFM_dc.tuw_flood_extent.where(GFM_dc.tuw_flood_extent!=255).compute()

### Plot



In [ ]:
band_name = "tuw_flood_extent"
print(f"Visualizing band: {band_name}")

GFM_dc[band_name].hvplot.image(x="x", y="y", title=band_name, cmap=["#0000FF19", "darkred"])

